### Question 1 - (15 points)
#### Problem formulation

The popularity of deep learning methods for genomic applications has grown impressively over the past few years. While not quite to the level of computer vision, both computational methods and user APIs have been developed specifically for the application to DNA (and protein) sequence data. Recall in such cases each input observation is a string of nucleotides (or amino acids in the protein example). Although it is indeed interesting to incorporate additional streams of data, we will restrict ourselves to only using the nucleotide sequence and then use CNNs to learn about the sequences. 

Specifically, we will be using a simulated dataset to investigate a common use scenario which may arise from ChIP-seq experiments. The goal of a [ChIP-seq](https://en.wikipedia.org/wiki/ChIP-sequencing) experiment is to use DNA sequencing to identify DNA-binding sites of proteins/transcription factors (TFs) along the genome. These proteins have varying effects on the genome, such as promoting or suppressing gene expression or even determining how the genome is folded and thus which parts are accessible to transcription. The downstream consequences of differential binding due to these proteins across healthy and diseased individuals is an active area of research. All simulations were generated via the [learning sequence motifs](https://github.com/p-koo/learning_sequence_motifs) package.

The details of the protocol are not important for this assignment. All you really need to know  is that each protein is relatively small (on the order of 6-18 base pairs in length) and exhibits preferential binding to certain consensus sequences. You may think of a single protein/TF somewhat like a lock and key: the protein preferentially binds (attaches) to the genome at locations where the pattern matches. ChIP is then used to locate those areas and DNA high-throughput sequencing yields the nucleotide sequences which were attached to the protein and for which we may then use as our input data. 

The consensus sequence that each protein looks for is referred to as a motif. Nine motifs are included in the simulation dataset and are provided below for reference. Conceptually, for a given motif pictured below, you may think of the height of the letter at a given position as the specificity of the protein binding to that nucleotide. For example, for the FOSL1 motif, the protein would almost exclusively bind to the pattern **TGA_TCA** where the underscore is either a C or a G with equal probability. ![Embedded motifs](img/hw_3_motifs.png)

We will consider the case in which our collaborators have performed nine different ChIP-seq experiments, each time *ChIP-ing* for a different protein (i.e. one of each protein from above), and provided us with a data file containing 200bp-length sequences called from each experiment. These are referred to as ChIP-seq peaks. For example, for the FOSL1 ChIP-seq experiment, the sequences called will contain the FOSL1 motif at some location within the 200 base-pairs. The actual nucleotide sequence of the realized motif in a given sequence will likely contain **TGA_TCA** where the underscore is either a C or a G with equal probability (note how this relates to above!). Additionally, a single peak may have been called in multiple experiments (Hint: class labels are not exclusive, consider how this will affect your loss function). Several example sequences are provided below, with the embedded motif visualized at its respective location within the observation. All other locations along the sequence are random genome background. Ideally our model will be able to learn these patterns.
![Example sequences](img/hw_3_grammers.png)

All reads (the nucleotide sequences) for the training, testing, and validation data have been joined in one file and the associated labels are provided as well. All data are stored in the hw_3_dataset.h5 object available on the canvas site. We will be training a CNN using the nucleotide sequences as input and the protein ChIP-ed for as the output label. We will then investigate what our model has learned. 

#### Train a CNN on simulation data

Load the libraries.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
import matplotlib.pyplot as plt
import keras
import h5py

Load the data. Note that we will not be using a data generator as we have in the past for image data. Therefore we may simply load each as a numpy array and then join them into a single dictionary. Each observation is already one-hot encoded.

In [ ]:
# load dataset


We need to permute the data arrays for keras to accept them. 

In [ ]:
# permute training, validation, and testing set


How many observations are there in the training set? What is the length of each sequence? What does the last channel (fourth dimension) of a given batch of observation correspond to?

In [ ]:
# get data shapes


What is the nucleotide sequence for the 100th observation? Hint: the column ordering is A, C, G, T. 

In [ ]:
# convert back to a nucleotide sequence


What is the output shape? Why or why not does this make sense considering the problem formulation above? 

In [ ]:
# calculate shapes


Now code up a CNN. Select the appropriate activation function for the last layer. Consider including regularization and pooling. 

In [ ]:
# build cnn
from keras import layers
from keras import models


Compile your model selecting the appropriate loss function and metric.

In [ ]:
# compile model
from keras import optimizers


Train your model for a suitable number of epochs.

In [ ]:
# fit model


Plot the training and validation accuracy and loss. How can you tell your model was trained for long enough? Provide an approximate epoch number denoting which training step you would choose for your final model.

In [ ]:
# Plot training & validation accuracy values

# Plot training & validation loss values


Save the model as a .h5 file.

In [ ]:
# save model


### Question 2 - (10 points)
#### Understanding what your model has learned

CNNs are often described as black boxes. However, that is not necessarily the case. Various methods have been developed which open up the black box. In class we looked at using the activation values from the convolutional filters and interpreting them as feature learners. In this question you will be using backpropagation methods, also referred to as saliency maps, to peer inside the box.

Begin by installing deeplift with your package manager (i.e. pip install deeplift). Import the module. Please see [deeplift](https://github.com/kundajelab/deeplift) for additional instructions and the [paper](https://arxiv.org/abs/1704.02685) for details on the algorithm.

In [ ]:
import deeplift
from deeplift.conversion import kerasapi_conversion as kc

Load the model saved in Question 1. Use the `kc.convert_model_from_saved_files` function to convert the model into the necessary format for deeplift and use default options provided in lab (`nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault`).

In [ ]:
# import and convert model


Print the layers of your model using the `.get_layers()` method.

In [ ]:
# print layers


Using the output from the previous question, determine the index for the input layer. This will be the layer you would like to view the importance scores for. In this homework please choose the input layer but in your own research any layer may be used so feel free to experiment with others! 

Also determine the index for the pre-activation output layer. Hint: This will almost surely be a dense layer. In python you may index to the last item in a list by indexing with -1. How would you get to the second to last item? 

In [ ]:
# define input and output layers


Now use the `get_target_contribs_func` method to calculate the contribution function for your input and output layers. This uses the model architecture of your CNN to define a function which may be used to calculate the importance scores for given observations and tasks (output channels). 

In [ ]:
# calculate contribution function


Lastly, calculate the feature importance scores for a given `task_idx` (the output class). Specify your batch size and input data. Why is it important to use your testing data?

In [ ]:
# calculate importance scores


And finally, visualize the importance scores for a given sequence. Make sure the sequence you visualize contains the motif corresponding to the `task_idx` you specified above. What do you notice about what your model has learned?

In [ ]:
# visualize scores


Extra credit: Repeat the above questions but use either 
    1. A different reference observation (e.g. random genome background, random sequence from test set, or anything else you think might be interesting)
    2.  A different method for computing importance scores (e.g. replace the `nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault` with a different mode)
It may be useful to view the documentation and genomics tutorial located [here](https://github.com/kundajelab/deeplift/blob/master/examples/genomics/genomics_simulation.ipynb). 

How does this impact your model interpretation? Can you provide a reason as to why this would be? 

In [ ]:
# Extra credit